# Logistic regression

로지스틱 회귀는 이름은 회귀이지만 분류 모델이다. 이 알고리즘은 선형회귀와 동일하게 선형 방정식을 학습한다. 그러나 이 최종값이 0과 1사이인 확률로 변경이 되어야 하는데, 이를 해결하는 방법이 로지스틱 함수이다. 로지스틱 함수는 시그노이드 함수라고도 불리우는데, 이 값은 선형 방정식을 통해 값이 양이 무한대에 가까워지면 1에 가까워지고 음에 무한대로 가까워지면 0에 가까워진다.

## 로지스틱 회귀로 2진 분류를 수행해보자.
물고기 데이터를 불러왔다. Weight, length, diagona, height, width를 해 species를 예측해보자.

In [1]:
import pandas as pd 

fish = pd.read_csv('https://bit.ly/fish_csv')
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


이제 이 데이터를 Train/Test set으로 나눠보자.

In [2]:
from sklearn.model_selection import train_test_split

fish_input = fish[['Weight', 'Length', 'Diagonal', 'Height', 'Width']].to_numpy()
fish_target = fish['Species'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(fish_input, fish_target, random_state = 42)

이제 사이킷런의 standard 클래스를 사용해 train set와 test set를 표준화 시켜주자.

In [4]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [8]:
train_input

array([[7.20000e+02, 3.50000e+01, 4.06000e+01, 1.63618e+01, 6.09000e+00],
       [5.00000e+02, 4.50000e+01, 4.80000e+01, 6.96000e+00, 4.89600e+00],
       [7.50000e+00, 1.05000e+01, 1.16000e+01, 1.97200e+00, 1.16000e+00],
       [1.10000e+02, 2.20000e+01, 2.35000e+01, 5.52250e+00, 3.99500e+00],
       [1.40000e+02, 2.07000e+01, 2.32000e+01, 8.53760e+00, 3.29440e+00],
       [6.90000e+01, 1.82000e+01, 2.03000e+01, 5.29830e+00, 2.82170e+00],
       [1.10000e+02, 2.10000e+01, 2.25000e+01, 5.69250e+00, 3.55500e+00],
       [6.20000e+02, 3.45000e+01, 3.97000e+01, 1.55227e+01, 5.28010e+00],
       [1.30000e+02, 2.13000e+01, 2.28000e+01, 6.38400e+00, 3.53400e+00],
       [8.50000e+01, 2.00000e+01, 2.10000e+01, 5.08200e+00, 2.77200e+00],
       [6.85000e+02, 3.65000e+01, 3.90000e+01, 1.08810e+01, 6.86400e+00],
       [5.00000e+02, 3.10000e+01, 3.62000e+01, 1.43714e+01, 4.81460e+00],
       [5.14000e+02, 3.28000e+01, 3.40000e+01, 1.00300e+01, 6.01800e+00],
       [2.00000e+02, 2.30000e+01, 2.58

In [10]:
train_scaled

array([[ 9.19657820e-01,  6.09431747e-01,  8.10412209e-01,
         1.85194896e+00,  1.00075672e+00],
       [ 3.00412188e-01,  1.54653445e+00,  1.45316551e+00,
        -4.69816625e-01,  2.72917446e-01],
       [-1.08585360e+00, -1.68646987e+00, -1.70848587e+00,
        -1.70159849e+00, -2.00447580e+00],
       [-7.97341433e-01, -6.08801762e-01, -6.74869070e-01,
        -8.24805885e-01, -2.76314705e-01],
       [-7.12898847e-01, -7.30625113e-01, -7.00926637e-01,
        -8.02298036e-02, -7.03386897e-01],
       [-9.12746301e-01, -9.64900787e-01, -9.52816444e-01,
        -8.80171862e-01, -9.91535662e-01],
       [-7.97341433e-01, -7.02512032e-01, -7.61727625e-01,
        -7.82824547e-01, -5.44530184e-01],
       [ 6.38182533e-01,  5.62576612e-01,  7.32239510e-01,
         1.64473401e+00,  5.07057367e-01],
       [-7.41046376e-01, -6.74398951e-01, -7.35670058e-01,
        -6.12059278e-01, -5.57331377e-01],
       [-8.67710255e-01, -7.96222301e-01, -8.92015456e-01,
        -9.33586942e-01

이제 우리가 불러온 데이터에서 도미(bream)와 빙어(smelt)의 행만 골라내자.

In [11]:
bream_smelt_indexes = (train_target == 'Bream') | (train_target == 'Smelt')
train_bream_smelt = train_scaled[bream_smelt_indexes]
target_bream_smelt = train_target[bream_smelt_indexes]

이제 이 데이터로 로지스틱 회귀 모델을 훈련해보자.
LogisticRegression 클래스는 선형 모델이므로 sklearn.linear_model 을 사용하면 된다.

In [12]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(train_bream_smelt, target_bream_smelt)

LogisticRegression()

다섯개 샘플을 예측해보자.

In [13]:
print(lr.predict(train_bream_smelt[:5]))

['Bream' 'Smelt' 'Bream' 'Bream' 'Bream']


In [15]:
target_bream_smelt[:5]

array(['Bream', 'Smelt', 'Bream', 'Bream', 'Bream'], dtype=object)

잘 예측이 되었다. 이제 예측 확률을 알아보자.

In [16]:
print(lr.predict_proba(train_bream_smelt[:5]))

[[0.99759855 0.00240145]
 [0.02735183 0.97264817]
 [0.99486072 0.00513928]
 [0.98584202 0.01415798]
 [0.99767269 0.00232731]]


이제 최종적으로 도미와 빙어를 예측하는 분류를 알아보았다.